In [82]:
import math,numpy,json,re,nltk
import time,re,os.path,sys
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from collections import defaultdict
from numpy import multiply
from math import sqrt
from nltk.tokenize import word_tokenize
from nltk.tag import StanfordNERTagger
from nltk.tokenize import word_tokenize
import os.path as path
from collections import OrderedDict
from nltk.stem.wordnet import WordNetLemmatizer as WNL
from sklearn.feature_extraction import DictVectorizer
from nltk import FreqDist, DictionaryProbDist
from operator import add
import csv, ast
from nltk.tag.stanford import StanfordPOSTagger
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.parse.stanford import StanfordDependencyParser
import string
path_to_jar = 'data/stanford-parser.jar'
path_to_models_jar = 'data/stanford-parser-3.7.0-models.jar'
st=PorterStemmer()
stop = set(stopwords.words('english'))
json_data = []
json_dataPOS = []
json_dataPOSOrg = []
dictDoc = {}
dictDocOrg = {}

In [83]:
def json_load_byteified(file_handle):
    return _byteify(json.load(file_handle, object_hook=_byteify),ignore_dicts=True)

def _byteify(data, ignore_dicts = False):
    # if this is a unicode string, return its string representation
    if isinstance(data, unicode):
        return data.encode('utf-8')
    # if this is a list of values, return list of byteified values
    if isinstance(data, list):
        return [ _byteify(item, ignore_dicts=True) for item in data ]
    # if this is a dictionary, return dictionary of byteified keys and values
    # but only if we haven't already byteified it
    if isinstance(data, dict) and not ignore_dicts:
        return {
            _byteify(key, ignore_dicts=True): _byteify(value, ignore_dicts=True)
            for key, value in data.iteritems()
        }
    # if it's anything else, return it in its original form
    return data

In [84]:
def num_there(s):
    return any(i.isdigit() for i in s)

def isfloat(value):
    try:
        float(value)
        return True
    except ValueError:
        return False

def importNERPOSQA(NER,POS,QA):
    global json_data, json_dataPOS, json_dataOrg, json_dataPOSOrg, dictDoc, dictDocOrg
    with open(NER) as json_file:
        json_data = json_load_byteified(json_file)    
    with open(POS) as json_file:
        json_dataPOS = json_load_byteified(json_file)
    with open(QA) as json_file:
        json_dataOrg = json_load_byteified(json_file)
    with open(POS) as json_file:
        json_dataPOSOrg = json_load_byteified(json_file)
    for document in range(len(json_data)):
        for thing in range(len(json_data[document])):
            for sentence in range(len(json_data[document][thing])):
                for word in range(len(json_data[document][thing][sentence])):
                    json_data[document][thing][sentence][word][1] = 'U' if word!=0 and json_data[document][thing][sentence][word][0][0].isupper() and json_data[document][thing][sentence][word][1]=='O' else json_data[document][thing][sentence][word][1]
                    json_data[document][thing][sentence][word][0] = json_data[document][thing][sentence][word][0].lower()
                    if json_data[document][thing][sentence][word][0] in ['one','two','three','four','five','six','seven','eight','nine','ten','zero'] or isfloat(json_data[document][thing][sentence][word][0]):
                        json_data[document][thing][sentence][word][1] = 'NUMBER'
    for document in range(len(json_dataPOS)):
        for thing in range(len(json_dataPOS[document])):
            for sentence in range(len(json_dataPOS[document][thing])):
                for word in range(len(json_dataPOS[document][thing][sentence])):
                    json_dataPOS[document][thing][sentence][word][0] = json_dataPOS[document][thing][sentence][word][0].lower()
                    if num_there(json_dataPOS[document][thing][sentence][word][0]):
                        json_dataPOS[document][thing][sentence][word][1] = 'CD'
    print 'import success'

In [115]:
def detectQuestion(i, j, k, l):
    global json_data, json_dataPOS, json_dataOrg, json_dataPOSOrg, dictDoc, dictDocOrg
    openclassword=[]
    kindOfAnswer = []
    questionPOS = json_dataPOS[i][0][j]
    questionNER = json_data[i][0][j]        
    answerPOS = json_dataPOS[i][l][k]
    answerNER = json_data[i][l][k]
    specialcommand=[]
    #print questionNER, answerPOS
    originalWithout = [x[0] for x in questionPOS]
    originalWithoutA = [x[0] for x in answerPOS]
    getIndexOfWH = [y for y,x in enumerate(questionPOS) if 'W' in x[1]]
    #if wh
    if len(getIndexOfWH) != 0:
        getPOS = ''.join([x[1][0] for x in questionPOS])        
        
        #not at last word
        if getIndexOfWH[0]+1!=len(questionPOS):            
            searchWordAfterWh1 = re.search('W(.*?)V', getPOS, re.IGNORECASE)
            if searchWordAfterWh1:
                if len(searchWordAfterWh1.group(1))!=0:
                    openclassword.append([searchWordAfterWh1.start()+len(searchWordAfterWh1.group(1))])                    
                else:
                    openclassword.append([])
            else:
                searchWordAfterWh1 = re.search('W(.*?)', getPOS, re.IGNORECASE)
                if searchWordAfterWh1:
                    openclassword.append([searchWordAfterWh1.start()+len(searchWordAfterWh1.group(0))-1])
                else:
                    openclassword.append([])
            frontPart = range(0,searchWordAfterWh1.start()) if searchWordAfterWh1 else range(0,len(getPOS)/2)
            backPart = range(searchWordAfterWh1.start()+len(searchWordAfterWh1.group(0)),len(getPOS)) if searchWordAfterWh1 else range(len(getPOS)/2,len(getPOS))                      
            openclassword.append(frontPart+backPart)      
            #print openclassword
        else:
            openclassword=[[],[y for y,x in enumerate(questionPOS)][:-1]]                        
        #remove Stopwords        
        openclassword[1] = [x for x in openclassword[1] if questionPOS[x][0] not in stop]
        if len(openclassword[0])!=0:            
            numberIndicator = ['year','length','percentage', 'many','much']            
            for x in numberIndicator: 
                for y in originalWithout[getIndexOfWH[0]:openclassword[0][0]+2]:
                    if x in y:
                        kindOfAnswer = ['NUMBER'] if len(kindOfAnswer)==0 else kindOfAnswer
                        specialcommand.append(x)
            personIndicator = ['name']
            for x in personIndicator:
                for y in originalWithout[getIndexOfWH[0]:openclassword[0][0]+2]:
                    if x in y:
                        kindOfAnswer = ['PERSON','ORGANIZATION'] if len(kindOfAnswer)==0 else kindOfAnswer
            placeIndicator = ['location','place','country','city','area']
            for x in placeIndicator:
                for y in originalWithout[getIndexOfWH[0]:openclassword[0][0]+2]:
                    if x in y:
                        kindOfAnswer = ['LOCATION'] if len(kindOfAnswer)==0 else kindOfAnswer
                        specialcommand=['location']
            if len(kindOfAnswer)==0:    
                kindOfAnswer = ['O','U']
        else:
            numberIndicator = ['when']            
            personIndicator = ['who','whom','whose','name']
            placeIndicator = ['where','located']
            if originalWithout[getIndexOfWH[0]] in numberIndicator:
                kindOfAnswer = ['NUMBER']
                specialcommand = ['year']
            elif originalWithout[getIndexOfWH[0]] in personIndicator:
                kindOfAnswer = ['PERSON','ORGANIZATION']
            elif originalWithout[getIndexOfWH[0]] in placeIndicator:
                kindOfAnswer = ['LOCATION']
                specialcommand=['location']
            else:
                if len([y for y in openclassword[1] if questionPOS[y][0] in personIndicator])!=0:
                    kindOfAnswer = ['PERSON','ORGANIZATION']
                elif len([y for y in openclassword[1] if questionPOS[y][0] in placeIndicator])!=0:
                    kindOfAnswer = ['LOCATION']
                    specialcommand=['location']
                else: 
                    kindOfAnswer = ['O','U']
    else:
        openclassword = [[],[x for x,y in enumerate(originalWithout) if y not in stop]]
        kindOfAnswer = ['O','U']
    #determine whether it requires number entity
    newList1=[]
    newList2=[]
    for x in range(len(openclassword[0])):
        if originalWithout[openclassword[0][x]] in originalWithoutA:            
            newList1.extend([f for f,h in enumerate(originalWithoutA) if originalWithout[openclassword[0][x]] == h])        
    for x in range(len(openclassword[1])):
        if originalWithout[openclassword[1][x]] in originalWithoutA:
            newList2.extend([f for f,h in enumerate(originalWithoutA) if originalWithout[openclassword[1][x]] == h])
    openclassword[0]=newList1
    openclassword[1]=newList2
    return openclassword, kindOfAnswer,specialcommand   
                    
def tokenizeUnicode(aUnicode):
    return word_tokenize(re.sub(',', '',re.sub('[^a-zA-Z0-9-_*., ]', ' ',aUnicode)))

def removeUnicode(aUnicode):
    return re.sub(',', '',re.sub('[^a-zA-Z0-9-_*., ]', ' ',aUnicode))

In [99]:
def readCSV(filename,l):
    global json_data, json_dataPOS, json_dataOrg, json_dataPOSOrg, dictDoc, dictDocOrg
    csv_file = filename
    i=0
    answerSecondFilter = []
    dictDoc={}
    dictDocOrg={}
    question={}
    with open(csv_file, 'rb') as csvfile:
        readCSV = csv.DictReader(csvfile, delimiter=',')
        for row in readCSV:
            document_i = int(row['document_ID'])
            question_i = int(row['question_ID'])
            filteredlistOfNERSentence = []
            question_type = []
            predictionList = ast.literal_eval(row['prediction_ID'])
            #try:
            question_type = detectQuestion(document_i,question_i,predictionList[0] if len(predictionList)!=0 else 1,l) 
            
            #except: print document_i,question_i,predictionList[0] if len(predictionList)!=0 else 1
            question[document_i,question_i] = row['question']        
            dictDoc[document_i,question_i]=predictionList[0] if len(predictionList)!=0 else 1,question_type

In [100]:
def returnAnswer(i,j,l):
    global json_data, json_dataPOS, json_dataOrg, json_dataPOSOrg, dictDoc, dictDocOrg
    answerListPOS = json_dataPOS[i][l][dictDoc[i,j][0]]    
    answerListNER = json_data[i][l][dictDoc[i,j][0]]    
    question_type = dictDoc[i,j][1][1]
    #print answerListPOS,question_type
    question_typeLocationInAnswer = [x for x,y in enumerate(answerListNER) if ('CD' in answerListPOS[answerListNER.index(y)][1] or 'NN' in answerListPOS[answerListNER.index(y)][1]) and y[1] in question_type and y[0] not in stop and x not in dictDoc[i,j][1][0][0] and x not in dictDoc[i,j][1][0][1]]
    #print question_typeLocationInAnswer
    if len(question_typeLocationInAnswer) ==0:
        question_typeLocationInAnswer = [x for x,y in enumerate(answerListNER) if ('CD' in answerListPOS[answerListNER.index(y)][1] or 'NN' in answerListPOS[answerListNER.index(y)][1]) and y[0] not in stop and x not in dictDoc[i,j][1][0][0] and x not in dictDoc[i,j][1][0][1]]
    #print question_typeLocationInAnswer,dictDoc[i,j][1][0][1], dictDoc[i,j][1][0][0]
    scoreList = 0
    if len(dictDoc[i,j][1][0][0])!=0:
        maxScore = sys.maxint        
        for x in question_typeLocationInAnswer:
            score = sum([math.fabs(z-x) for z in dictDoc[i,j][1][0][0]])
            if maxScore>score:
                maxScore=score
                scoreList = x
    else:
        maxScore = sys.maxint
        scoreList = 0
        for x in question_typeLocationInAnswer:
            score = sum([math.fabs(z-x) for z in dictDoc[i,j][1][0][1]])
            if maxScore>score:
                maxScore=score
                scoreList = x
    #print json_dataPOSOrg[i][2][dictDoc[i,j][0]][scoreList][0]
    #print dictDoc[i,j][1][2]
    return createNP((json_dataPOSOrg[i][l][dictDoc[i,j][0]][scoreList][0],scoreList),json_dataPOSOrg[i][l][dictDoc[i,j][0]],dictDoc[i,j][1][2],i,dictDoc[i,j][0])

In [127]:
def createNP(answerToReturn,answerPOS,specialcommand,q,j):
    global json_data, json_dataPOS, json_dataOrg, json_dataPOSOrg, dictDoc, dictDocOrg
    newAnswer = answerPOS[answerToReturn[1]][0]
    if (newAnswer.isdigit() and 'year' not in specialcommand):
        newAnswer = "{:,}".format(int(answerPOS[answerToReturn[1]][0]))
    for i in range(answerToReturn[1],0,-1):
        if answerToReturn[1] != 0:
            if answerPOS[i][1] =='NNP':
                if 'NNP' in answerPOS[i-1][1] or (('DT' in answerPOS[i-1][1] or 'IN' in answerPOS[i-1][1]) and answerPOS[i-1][0] !='at' and 'location' in specialcommand):
                    newAnswer = answerPOS[i-1][0]+ " " +newAnswer
                else:
                    break
            elif answerPOS[i][1] =='NN':
                if 'JJ' in answerPOS[i-1][1] or 'DT' in answerPOS[i-1][1] or 'CD' in answerPOS[i-1][1] or answerPOS[i-1][1] =='NN':
                    newAnswer = answerPOS[i-1][0]+ " " +newAnswer
                else:
                    break
            elif answerPOS[i][1] =='JJ':
                if answerPOS[i-1][1] =='JJ':
                    newAnswer = answerPOS[i-1][0]+ " " +newAnswer
                else:
                    break
            elif answerPOS[i][1] =='NNS':
                if 'JJ' in answerPOS[i-1][1]:
                    newAnswer = answerPOS[i-1][0]+ " " +newAnswer
                else:
                    break
            elif 'DT' in answerPOS[i][1] and 'location' not in specialcommand:
                if 'TO' in answerPOS[i-1][1]:
                    newAnswer = answerPOS[i-1][0]+ " " +newAnswer
                else:
                    break
            else:
                if answerPOS[i][1] !='CD' and answerPOS[i][1] !='RB':
                    if 'NN' in answerPOS[i-1][1] or 'JJ' in answerPOS[i-1][1] or 'RB' in answerPOS[i-1][1]:
                        newAnswer = answerPOS[i-1][0]+ " " +newAnswer
                    else:
                        break
                else:
                    break
        else:
            break
    for i in range(answerToReturn[1],len(answerPOS)-1):
        if answerToReturn[1] != len(answerPOS)-1:
            if answerPOS[i][1] =='NNP':                
                if 'NNP' in answerPOS[i+1][1] or 'CC' in answerPOS[i+1][1] or 'IN' in answerPOS[i+1][1] or 'TO' in answerPOS[i+1][1] or ('NN' in answerPOS[i+1][1] and 'location' in specialcommand):                                        
                    newAnswer = newAnswer+' '+answerPOS[i+1][0]
                else:                    
                    break
            elif answerPOS[i][1] =='CC':
                if 'NNP' in answerPOS[i+1][1] or 'NN' == answerPOS[i+1][1]:
                    newAnswer = newAnswer+' '+answerPOS[i+1][0]
                else:                    
                    break
            elif answerPOS[i][1] =='CD':
                if 'CD' in answerPOS[i+1][1]:                 
                    newAnswer = newAnswer+' '+answerPOS[i+1][0]
                else:                    
                    break
            elif answerPOS[i][1] =='TO' or answerPOS[i][1] =='DT':
                if 'NN' in answerPOS[i+1][1] or 'RB' in answerPOS[i+1][1]:
                    newAnswer = newAnswer+' '+answerPOS[i+1][0]
                else:
                    break 
            elif answerPOS[i][1] =='IN' and 'location' not in specialcommand:
                if answerPOS[i+1][1] =='DT' or answerPOS[i+1][1] =='NNP' or answerPOS[i+1][1] =='TO' or answerPOS[i+1][1] =='CD':
                    newAnswer = newAnswer+' '+answerPOS[i+1][0]
                else:
                    break
            elif answerPOS[i][1] =='JJ':
                if 'NNS' in answerPOS[i+1][1] or 'NN' == answerPOS[i+1][1] or answerPOS[i+1][1] =='JJ':
                    newAnswer = newAnswer+' '+answerPOS[i+1][0]
                else:
                    break
            elif answerPOS[i][1] =='NN':
                if 'NN' in answerPOS[i+1][1] or 'JJ' in answerPOS[i+1][1] or 'IN' in answerPOS[i+1][1] or 'CC' in answerPOS[i+1][1]:
                    newAnswer = newAnswer+' '+answerPOS[i+1][0]
                else:
                    break
            elif answerPOS[i][1] =='NNS':
                if 'IN' in answerPOS[i+1][1] and 'ORG' in specialcommand:
                    newAnswer = newAnswer+' '+answerPOS[i+1][0]
                else:
                    break
            else:
                break
        else:
            break
    buffernewAnswer1 = newAnswer.split()
    if buffernewAnswer1[-1]=='along' and len(buffernewAnswer1)>1:
        newAnswer=newAnswer[:-6]
    if buffernewAnswer1[-1]=='and' and len(buffernewAnswer1)>1:
        newAnswer=newAnswer[:-4]
    if buffernewAnswer1[-1]=='for' and len(buffernewAnswer1)>1:
        newAnswer=newAnswer[:-4]
    if buffernewAnswer1[-1]=='but' and len(buffernewAnswer1)>1:
        newAnswer=newAnswer[:-4]
    if buffernewAnswer1[-1]=='in' and len(buffernewAnswer1)>1:
        newAnswer=newAnswer[:-3]
    if buffernewAnswer1[-1]=='as' and len(buffernewAnswer1)>1:
        newAnswer=newAnswer[:-3]
    if buffernewAnswer1[-1]=='at' and len(buffernewAnswer1)>1:
        newAnswer=newAnswer[:-3]
    if buffernewAnswer1[-1]=='on' and len(buffernewAnswer1)>1:
        newAnswer=newAnswer[:-3]
    if buffernewAnswer1[-1]=='to' and len(buffernewAnswer1)>1:
        newAnswer=newAnswer[:-3]
    if buffernewAnswer1[-1]=='while' and len(buffernewAnswer1)>1:
        newAnswer=newAnswer[:-6]
    if buffernewAnswer1[-1]=='despite' and len(buffernewAnswer1)>1:
        newAnswer=newAnswer[:-8]
    if buffernewAnswer1[-1]=='because' and len(buffernewAnswer1)>1:
        newAnswer=newAnswer[:-8]
    if buffernewAnswer1[-1].isdigit() and len(buffernewAnswer1[-1])==4 and 'year' in specialcommand:
        newAnswer=buffernewAnswer1[-1]
    if buffernewAnswer1[-1].isdigit() and 'location' in specialcommand:
        newAnswer=buffernewAnswer1[-1]
    if len(specialcommand)!=0 and 'percentage' in specialcommand[0]:
        newAnswer+='%'
    if 'location' in specialcommand:
        bufferNewAnswer=[]        
        for x in newAnswer.split():
            for y in json_dataOrg[q]['sentences'][j].split():
                if x in y:
                    bufferNewAnswer.append(y)
                    break
        newAnswer=' '.join(bufferNewAnswer)
        newAnswer = newAnswer[:-1] if (len(newAnswer)>1 and not (newAnswer[-1].isalnum() or newAnswer[-1]=='%')) else newAnswer
    newAnswer = newAnswer.replace(',','-COMMA-')
    return newAnswer

In [123]:
def writeToFile(filename,l):
    global json_data, json_dataPOS, json_dataOrg, json_dataPOSOrg, dictDoc, dictDocOrg
    with open(filename, mode='wb',) as csv_file:
        fieldnames = ['document_id','question_id','answer_predict',"answer_actual",'tag','sentence','predict','question_type','question'] if l==2 else ['id','answer']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames,delimiter=',')
        writer.writeheader()
        k = 0
        totalDoc=len(json_data)
        for i in range(0, totalDoc):
            for j in range(0,len(json_data[i][0])):
                k+=1            
                dictToCSV={}
                if l == 2:
                    dictToCSV['document_id'] = i
                    dictToCSV['question_id'] = j
                    dictToCSV['answer_predict']= returnAnswer(i,j,l)                    
                    dictToCSV['answer_actual'] = json_dataOrg[i]['qa'][j]['answer']
                    dictToCSV['tag'] =dictDoc[i,j]
                    dictToCSV['question_type'] = dictDoc[i,j][1][1] if len(dictDoc[i,j][1])>=2 else 'O'
                    dictToCSV['question'] = json_dataOrg[i]['qa'][j]['question']
                    dictToCSV['sentence'] = json_dataOrg[i]['qa'][j]['answer_sentence']
                    dictToCSV['predict'] = dictDoc[i,j][0]
                else:
                    dictToCSV={}
                    dictToCSV['id'] = k
                    #try:
                    dictToCSV['answer'] = returnAnswer(i,j,l)
                    #except:
                     #   print '\nError',i,j
                writer.writerow(dictToCSV)    
                csv_file.flush()
            sys.stdout.write('\r')
            sys.stdout.write(str(k))
            sys.stdout.flush()
    csv_file.close()
    print '\nsuccess' 

In [120]:
importNERPOSQA("data/NERtest.json","data/POStest.json","data/QA_test.json")
readCSV("data/bm25_test.csv",1)
writeToFile("data/bm25_test_result.csv",1)

import success
8974
success


In [121]:
importNERPOSQA("data/NERdev.json","data/POSdev.json","data/QA_dev.json")
readCSV("data/bm25_dev.csv",2)
writeToFile("data/bm25_dev_result.csv",2)

import success
8463
success


In [129]:
importNERPOSQA("data/NERtrain.json","data/POStrain.json","data/QA_train.json")
readCSV("data/tfidf_train.csv",2)
writeToFile("data/tfidf_train_result.csv",2)

import success
70159
success
